In [131]:
import os, sys
import re
from pymystem3 import Mystem
import pandas as pd
import numpy as np
import multiprocessing as mp
import urllib
import pickle
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from collections import Counter
from elasticsearch import Elasticsearch
from elasticsearch.helpers import parallel_bulk
import networkx as nx

In [2]:
def read_docs(path):
    df = pd.read_csv(path, skiprows=0)
    ids = df['id'].to_numpy()
    docs = df['text'].to_numpy()
    return ids, docs

ids, docs = read_docs('texts.csv')

In [120]:
settings = {
    'mappings': {
        'properties': {
            'content': {
                'type': 'text'
             }
         }
    },
    'settings': {
        'analysis': {
            'analyzer': {
                'white_lover': {
                    'tokenizer': 'letter',
                    'filter': [
                        'lowercase', 
                        'my_snow'
                    ]
                }
            },
            'filter': { 
                'my_snow': {
                    "type" : "snowball",
                    "language" : ["Russian", "English"]
                }
            }
        }
    }
}

es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'timeout': 360, 'maxsize': 25}])
es.indices.delete('myandex')
es.indices.create(index='myandex', body=settings)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'myandex'}

In [70]:
def create_es_action(index, doc_id, document):
    return {
        '_index': index,
        '_id': doc_id,
        '_source': document
    }

def create_index():
    def next_document():
        for i, doc in tqdm_notebook(list(zip(ids, docs))):
            if doc is not None:
                yield create_es_action('myandex', int(i), {'content': str(doc)})
    
    for ok, result in parallel_bulk(es, next_document(), queue_size=4, thread_count=4, chunk_size=1000):
        if not ok:
            print(result)

In [71]:
create_index()

In [24]:
es.indices.delete('myandex')

{'acknowledged': True}

In [105]:
def search(query, *args):
    pretty_print_result(es.search(index='myandex', body=query, size=20), args)
    # note that size set to 20 just because default value is 10 and we know that we have 12 docs and 10 < 12 < 20
                        
def pretty_print_result(search_result, fields=[]):
    # fields is a list of fields names which we want to be printed
    res = search_result['hits']
    print(f'Total documents: {res["total"]["value"]}')
    for hit in res['hits']:
        print(f'Doc {hit["_id"]}, score is {hit["_score"]}')
        for field in fields:
            print(f'{field}: {hit["_source"][field]}')
                  
def get_doc_by_id(doc_id):
    return es.get(index='myandex', id=doc_id)['_source']

query = {
    'query': {
        'bool': {
            'must': [
                {
                    'match': {
                        'content': 'Андрей'
                    }
                }
            ]
        }
    }
}

search(query, 'content')

Total documents: 9921
Doc 332329, score is 6.322534
content: Pressball.by :: Поиск
 Pressball.by :: Поиск Pressball.by Всё о спортивной жизни FAQ Пользователи Группы Регистрация Профиль Войти и проверить личные сообщения Вход Результатов поиска: 1808 Список форумов Pressball.by Автор Сообщение Тема: Тёзки Андрей Зюзя Ответов: 42 Просмотров: 247 Форум: Разговоры обо всем Добавлено: 23 Фев 2007 00:22 Тема: Тёзки 1812-е посвящаю Кутузову... Человеку и...человеку... Одноглазому и двуглазому... В общем обоим... Пойду в общем спать.... Хватит на сегодня Тема: Тёзки Андрей Зюзя Ответов: 42 Просмотров: 247 Форум: Разговоры обо всем Добавлено: 23 Фев 2007 00:19 Тема: Тёзки Значит так... 1802-е сообщение посвящаю denunited, а 1806-е посвящаю saxar1... Все довольны???? Следующий... Тема: Виталий Кутузов: "Перед Пунтусом как игрок я чист" Андрей Зюзя Ответов: 33 Просмотров: 289 Форум: Обсуждение статей Добавлено: 23 Фев 2007 00:17 Тема: Виталий Кутузов: "Перед Пунтусом как игрок я чист" Babangida,

In [109]:
def print_index_size(index): 
    print(f"{(es.indices.stats(index)['_all']['primaries']['store']['size_in_bytes'] / 2 ** 30):.2f} GB")

In [110]:
print_index_size('myandex')

1.48 GB


In [139]:
m = Mystem()

def lemmatize_doc(doc):
    return ''.join(m.lemmatize(str(doc)))
    
def lemmatize_collection(docs):
    result = []
    for doc in tqdm_notebook(docs):
        result.append(lemmatize_doc(doc))
    return result



In [141]:
def save_lemmas(lemmas):
    with open('lemmas.pickle', 'wb') as f:
        pickle.dump(lemmatized_docs, f)

def load_lemmas():
    with open('lemmas.pickle', 'rb') as f:
        return pickle.load(f)

In [142]:
# lemmatized_docs = lemmatize_collection(docs)
lemmatized_docs = load_lemmas()

логойск | портал - мир переживать самый теплый зима за весь история погодный наблюдение
 логойск | портал - мир переживать самый теплый зима за весь история погодный наблюдение запоминать? забывать пароль? регистрация iq150.team present! главный мир мир переживать самый теплый зима за весь история погодный наблюдение логойск online новость белорусский швейцария | главный | блог | поиск | о сайт | обратный связь | ресурс история логойск галерея погода форум архив файл локальный сеть сош№1 ссылка логойск достопримечательность литература карта район карта город расписание автобус википедия ретро фотофакт история логойск общий сведение наследие тышкевич археология земляк литература религия XX век палеский сон мир переживать самый теплый зима за весь история погодный наблюдение минуть зима в северный полушарие становиться самый теплый за последний 125 год, то быть за весь время, пока вестись климатический наблюдение. об это, как передавать ввс, заявлять эксперт национальный агентство сша по